### 需要的基础知识
- 前端相关知识
- Linux基础命令
- HTTP协议相关内容
- 网络相关内容（TCP/IP协议）
- 算法、数据结构
- 操作系统原理

_____
### urls.py

### views.py

### models.py

### forms.py

### templates文件夹

### admin.py

### setting.py

_____
### virtualenvwrapper虚拟环境管理
```
$ pip3.6 install virtualenv virtualenvwrapper   # 管理虚拟开发环境

$ mkvirtualenv zqxt  # 创建运行环境zqxt
$ workon zqxt    # 工作在 zqxt 环境 或 从其它环境切换到 zqxt 环境
$ deactivate   # 退出终端环境

$ rmvirtualenv ENV   # 删除运行环境ENV
$ lsvirtualenv   # 列出可用的运行环境

```

### 基本命令
```
$ django-admin.py startproject project_name    # 新建一个django project
$ cd project_name
$ python manage.py startapp app_name    # 进入project_name目录下运行，新建app，一般一个project有多个app
```
将新定义的app_name添加到settings.py中的```INSTALL_APPS```。若不添加，django就不能找到app_name中的**模板文件**和**静态文件**。


```
$ python manage.py makemigrations     # 创建更改的文件
$ python manage.py migrate       # 将生成的py文件应用到数据库

$ python manage.py runserver 8001     # 查看开发服务器

$ python manage.py flush      # 清空数据库

$ python manage.py createsuperuser    # 创建超级管理员

$ python manage.py shell      # 进入shell，可调用当前项目models.py中的API

$ python manage.py dbshell      # 进入数据库命令行
```

____
网址是写在urls.py文件中，用正则表达式对应views.py中的一个函数。

#### 定义视图函数
```Python
# coding:utf-8
# learn/views.py
from django.shortcuts import render
from django.http import HttpResponse

# Create your views here.
# 第一个参数必须是request，与网页发来的请求有关，包含get或post内容
def index(request):
	# HttpResponse：向网页返回内容，与print类似，只不过显示在网页上
	return HttpResponse(u"欢迎光临，自强学堂！")
```

##### 定义视图函数相关的URL，即访问什么网站对应什么内容
```Python
# mysite/mysite/urls.py
from django.conf.urls import url
from django.contrib import admin
from learn import views as learn_views

urlpatterns = [
	url(r'^$', learn_views.index),
    url(r'^admin/', admin.site.urls),
]
```

______
### 进阶
```
$ django-admin.py startproject zqxt_views
$ cd zqxt_views
$ python manage.py startapp calc
```

_____
```Python
# zqxt_views/calc/views.py
from django.shortcuts import render
from django.http import HttpResponse

# Create your views here.
def add(request):
	a = request.GET.get('a', 0)
	b = request.GET.get('b', 0)
	c = int(a) + int(b)
	return HttpResponse(str(c))

def add2(request, a, b):
	c = int(a) + int(b)
	return HttpResponse(str(c))
```

_____
```Python
# zqxt_views/zqxt_views/urls.py
from django.conf.urls import url
from django.contrib import admin
from calc import views as calc_views

urlpatterns = [
	# 访问地址形式为：http://127.0.0.1:8000/add/?a=4&b=5
	url(r'^add/$', calc_views.add, name='add'),
	# 访问地址形式为：http://127.0.0.1:8000/add/4/5/
	url(r'^add/(\d+)/(\d+)/$', calc_views.add2, name='add2'),
   url(r'^admin/', admin.site.urls),
]
```

_____

### URL name详解
- 修改 calc/views.py
```Python
def index(request):
	# 自动找到INSTALLED_APPS中对应app下的templates中的文件
	return render(request, 'home.html')
```

- 新加calc/templates/home.html

```html
<<!DOCTYPE html>
<html>
<head>
	<title>自强学堂</title>
</head>
<body>

<a href="/add/4/5">计算 4+5</a>

</body>
</html>
```

- 修改zqxt_views/urls.py

```Python
from calc import views as calc_views

urlpatterns = [
	url(r'^$', calc_views.index, name='home'),   
	# 访问地址形式为：http://127.0.0.1:8000/add/?a=4&b=5
	url(r'^add/$', calc_views.add, name='add'),
	# 访问地址形式为：http://127.0.0.1:8000/add/4/5/
	url(r'^add/(\d+)/(\d+)/$', calc_views.add2, name='add2'),
    url(r'^admin/', admin.site.urls),
]
```

```<a href="/add/4/5">计算 4+5</a>```：这样写死了网站，会使得改了网址后，templates、views.py、models.py同了此网址的，都必须进行相应的更改。

#### 优化方法
```
不带参数的：
{% url 'name' %}
带参数的：参数可以是变量名
{% url 'name' 参数 %}
```

``<a href="{% url 'add2' 4 5 %}">link</a>```，只要不改name，获取的网址也会动态地跟着变。

想改网址的时候，修改urls.py中的正则表达式部分即可。

_____
#### 如何让旧的/add/3/4自动跳转到新的网址
- 在views.py写一个跳转的函数

- calc/view.py
```Python
def old_add2_redirect(request, a, b):
	return HttpResponseRedirect(reverse('add2', args=(a, b)))
```

- zqxt_views/urls.py
```Python
urlpatterns = [
	url(r'^$', calc_views.index, name='home'),
	# 访问地址形式为：http://127.0.0.1:8000/add/?a=4&b=5
	url(r'^add/$', calc_views.add, name='add'),
	# 访问地址形式为：http://127.0.0.1:8000/add/4/5/
	url(r'^new_add/(\d+)/(\d+)/$', calc_views.add2, name='add2'),
	url(r'^add/(\d+)/(\d+)/$', calc_views.old_add2_redirect),
    url(r'^admin/', admin.site.urls),
]
```

_____
### 模板
做网站有一些通用的部分，比如导航nav.html、底部bottom.html、访问统计代码tonji.html等等。可以写在一个**base.html**来包含这些通用文件。
```html
<!DOCTYPE html>
<html>
    <head>
        <title>{% block title %}默认标题{% endblock %} - 自强学堂</title>
    </head>
    
    <body>
        {% block content %}
        <div>这里是默认内容，所有继承自这个模板的，如果不覆盖就显示这里的默认内容。</div>
        {% endblock %}
        
        {% include 'bottom.html' %}
        
        {% include 'tongji.html' %}
    </body>
</html>
```

- include是包含其他文件的内容，即把一些网页共用的部分拿出来，重复利用。
- 继承后的模板也可以在block块中include其它的模板文件。

如首页home.html，继承原来的base.html文件：
```
{% extends 'base.html' %}

{% block title %}欢迎光临首页{% endblock %}

{%block content %}
{%include 'ad.html' %}
这里是首页，欢迎光临
{% endblock %}
```

- 模板一般放在app下的templates中。Django查找模板的过程是在*每个*app的templates文件夹中找，存在的弊端是可能找错模板。
- 所以在使用的时候，在templates中建立一个app同名的文件夹， 仅和该app相关的模板放在app/templates/app目录下面。
- 此时，使用的时候，模板就是app/home.html。有app作为名称的一部分。